# Emission Rates Generation for ST Maps

## g/veh

In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# ==============================================================================
# 1. CONFIGURACIÓN DE RUTAS
# ==============================================================================

# Carpeta donde están los archivos de SALIDA de MOVES (.tab o .csv) (A, B, C)
MOVES_OUTPUT_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1\Input files\Summary Reports para ER Generation")

# Carpeta donde está tu archivo 'sourceTypePopulation.csv'
POPULATION_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1\Input files\MOVES UIDB Population Table")

# --- NUEVA RUTA DE SALIDA PARA LAS TASAS (DI) ---
OUTPUT_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1\Output Files\Generated emission rates")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Mapeo de nombres para facilitar lectura
SOURCE_TYPE_NAMES = {
    11: "Motorcycle", 21: "Passenger Car", 22: "Taxi", 31: "Passenger Truck",
    32: "Light Commercial Truck", 42: "Transit Bus", 43: "School Bus", 44: "Other Buses",
    51: "Refuse Truck", 52: "Single Unit Short-haul Truck",
    53: "Single Unit Long-haul Truck", 54: "Motor Home",
    61: "Combination Short-haul Truck", 62: "Combination Long-haul Truck"
}

# ==============================================================================
# 2. FUNCIONES DE CARGA
# ==============================================================================

def identify_and_load_moves_files(root_folder):
    """
    Busca archivos .tab o .csv, identifica si son A, B o C según sus columnas
    y los carga en un diccionario.
    """
    path_root = Path(root_folder)
    all_files = list(path_root.rglob("*.tab")) + list(path_root.rglob("*.csv"))
    
    dfs = {}
    print(f"📂 Buscando archivos de salida MOVES en: {root_folder}")
    
    for file_path in all_files:
        try:
            # Leer solo la primera fila para checar columnas
            if file_path.suffix == '.tab':
                sep = '\t'
            else:
                sep = ','
                
            df_temp = pd.read_csv(file_path, sep=sep, nrows=1)
            cols = df_temp.columns.tolist()
            
            # Lógica de identificación
            tag = None
            if "CO2_Equiv" in cols:
                tag = 'A'
            elif "NOx" in cols:
                tag = 'B'
            elif "TotalEnergy" in cols:
                tag = 'C'
            
            if tag:
                print(f"   -> Cargando {tag}: {file_path.name}")
                dfs[tag] = pd.read_csv(file_path, sep=sep)
                
        except Exception as e:
            print(f"⚠️ Error leyendo {file_path.name}: {e}")
            
    return dfs

def load_population_table(pop_dir):
    """Carga la tabla de población generada anteriormente."""
    pop_path = pop_dir / "sourceTypePopulation.csv"
    if not pop_path.exists():
        raise FileNotFoundError(f"❌ No se encuentra sourceTypePopulation.csv en {pop_dir}")
    
    print(f"📂 Cargando Población desde: {pop_path.name}")
    df_pop = pd.read_csv(pop_path)
    return df_pop[['sourceTypeID', 'sourceTypePopulation']]

# ==============================================================================
# 3. PROCESAMIENTO Y CÁLCULO DE TASAS
# ==============================================================================

def generate_rates_lookup(dfs_moves, df_pop):
    print("\n🔄 Unificando archivos A, B y C...")
    
    # 1. Merge de A, B y C using standard MOVES keys
    merge_keys = ['MOVESRunID', 'Year', 'Month', 'Day', 'Hour', 'Source', 'Fuel', 'ModelYr', 'Road']
    
    # Filtramos llaves que realmente existan en el DF A
    keys_in_A = [k for k in merge_keys if k in dfs_moves['A'].columns]
    
    df_merged = dfs_moves['A']
    if 'B' in dfs_moves:
        df_merged = pd.merge(df_merged, dfs_moves['B'], on=keys_in_A, how='inner')
    if 'C' in dfs_moves:
        df_merged = pd.merge(df_merged, dfs_moves['C'], on=keys_in_A, how='inner')

    print(f"   -> Filas totales unificadas: {len(df_merged):,}")

    # 2. Agrupación por Dia, Hora y Tipo Vehículo (Source)
    # Identificar columnas numéricas para sumar
    numeric_cols = df_merged.select_dtypes(include=[np.number]).columns.tolist()
    
    # Llaves de agrupación
    group_keys = ['Day', 'Hour', 'Source']
    
    # Columnas a sumar (excluyendo llaves y metadatos)
    cols_to_sum = [c for c in numeric_cols if c not in group_keys and c not in ['MOVESRunID', 'Year', 'Month', 'ModelYr']]
    
    print("🔄 Agrupando emisiones por [Day, Hour, Source]...")
    df_grouped = df_merged.groupby(group_keys)[cols_to_sum].sum().reset_index()

    # 3. Merge con Población
    print("🔄 Cruzando con tabla de Población...")
    df_final = pd.merge(df_grouped, df_pop, left_on='Source', right_on='sourceTypeID', how='left')
    
    # Validar match
    missing_pop = df_final[df_final['sourceTypePopulation'].isna()]
    if not missing_pop.empty:
        print(f"⚠️ ADVERTENCIA: {len(missing_pop)} filas no encontraron población (SourceIDs: {missing_pop['Source'].unique()})")
        df_final['sourceTypePopulation'] = df_final['sourceTypePopulation'].fillna(1) 

    # 4. Cálculo de Tasas (g/veh)
    print("🧮 Calculando tasas (g/veh)...")
    
    rate_cols = []
    for col in cols_to_sum:
        rate_col_name = f"Rate_{col}_per_veh"
        # Evitar división por cero
        df_final[rate_col_name] = df_final[col] / df_final['sourceTypePopulation']
        rate_cols.append(rate_col_name)

    # 5. Limpieza Final y FILTRADO DE COLUMNAS NO DESEADAS
    df_final['sourceTypeName'] = df_final['Source'].map(SOURCE_TYPE_NAMES).fillna("Unknown")
    
    # Definir columnas a EXCLUIR explícitamente
    cols_to_exclude = ['Rate_Fuel_per_veh', 'Rate_Road_per_veh', 'Rate_TotalEnergy_per_veh']
    
    # Organizar columnas
    final_cols = ['Day', 'Hour', 'Source', 'sourceTypeName', 'sourceTypePopulation'] + rate_cols
    
    # Filtro Final: Que existan en el DF Y que NO estén en la lista de exclusión
    final_cols = [c for c in final_cols if c in df_final.columns and c not in cols_to_exclude]
    
    return df_final[final_cols]

# ==============================================================================
# 4. EJECUCIÓN
# ==============================================================================

if __name__ == "__main__":
    # 1. Cargar Archivos MOVES
    moves_data = identify_and_load_moves_files(MOVES_OUTPUT_DIR)
    
    if 'A' in moves_data: # Mínimo necesitamos el A
        # 2. Cargar Población
        try:
            df_population = load_population_table(POPULATION_DIR)
            
            # 3. Generar Lookup Table
            df_rates = generate_rates_lookup(moves_data, df_population)
            
            # 4. Guardar como PARQUET
            output_parquet = OUTPUT_DIR / "Lookup_Table_Emission_Rates_g_per_veh.parquet"
            df_rates.to_parquet(output_parquet, index=False)
            
            print(f"\n✅ ¡ÉXITO! Lookup Table generada correctamente.")
            print(f"   (Se han excluido las columnas: Rate_Fuel, Rate_Road, Rate_TotalEnergy)")
            print(f"📂 Archivo guardado en: {output_parquet}")
            print("\nPrimeras 5 filas:")
            print(df_rates.head())
            
        except Exception as e:
            print(f"❌ Error en el proceso: {e}")
    else:
        print("❌ No se encontró el archivo A (o faltan archivos) en la carpeta indicada.")

📂 Buscando archivos de salida MOVES en: D:\NB3_VISUM plus MOVES merge\Integration Files V1\Input files\Summary Reports para ER Generation
   -> Cargando A: SummaryReportBody.tab
   -> Cargando B: SummaryReportBody.tab
   -> Cargando C: SummaryReportBody.tab
📂 Cargando Población desde: sourceTypePopulation.csv

🔄 Unificando archivos A, B y C...
   -> Filas totales unificadas: 143,040
🔄 Agrupando emisiones por [Day, Hour, Source]...
🔄 Cruzando con tabla de Población...
🧮 Calculando tasas (g/veh)...

✅ ¡ÉXITO! Lookup Table generada correctamente.
   (Se han excluido las columnas: Rate_Fuel, Rate_Road, Rate_TotalEnergy)
📂 Archivo guardado en: D:\NB3_VISUM plus MOVES merge\Integration Files V1\Output Files\Generated emission rates\Lookup_Table_Emission_Rates_g_per_veh.parquet

Primeras 5 filas:
   Day  Hour  Source          sourceTypeName  sourceTypePopulation  \
0    2     1      11              Motorcycle                   964   
1    2     1      21           Passenger Car               

## g/km

In [5]:
import pandas as pd
from pathlib import Path
import numpy as np
import time

# Iniciar cronómetro para medir rendimiento
start_time = time.time()

# ==============================================================================
# 1. CONFIGURACIÓN E INPUTS
# ==============================================================================
# Ruta principal
BASE_INPUT_DIR = Path(r"D:\MOVES\Sensitivity Analysis RS14\RS14\Sensitivity Analysis Files\All SBs")

# Rutas de salida
OUTPUT_PIVOTED_PARQUET = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1\Output Files\Generated emission rates\Lookup_Table_Emission_Rates_g_per_km.parquet")

# Claves para unir los archivos A, B y C
MERGE_KEYS = ['Day', 'Hour', 'Source', 'Fuel', 'ModelYr', 'Road']

# Mapa de contaminantes (Columnas MOVES -> Columnas Salida)
pollutant_map = {
    "CO2_Equiv":   "CO2_Equiv",
    "CO":          "CO",
    "CO2":         "CO2",
    "NOx":         "NOx",          # Archivo B
    "Total_PM10":  "PM10",         # Archivo B
    "Total_PM25":  "PM25",         # Archivo B
    "TotalHC":     "HC",           # Archivo C
    "TotalEnergy": "Energy"        # Archivo C
}

# ==============================================================================
# 2. CARGA Y FUSIÓN DE ARCHIVOS (A + B + C)
# ==============================================================================
print(f"--- FASE 1: CARGA DE DATOS ---")
print(f"Directorio base: {BASE_INPUT_DIR}")
print("Iniciando carga y unificación de 16 Speed Bins...")

all_dfs = []
bins_encontrados = []

for i in range(1, 17):
    sb_name = f"SB{i:02d}"
    sb_path = BASE_INPUT_DIR / sb_name
    
    # Rutas específicas
    path_a = sb_path / "A" / "SummaryReportBody.tab"
    path_b = sb_path / "B" / "SummaryReportBody.tab"
    path_c = sb_path / "C" / "SummaryReportBody.tab"
    
    # Verificar existencia
    if not (path_a.exists() and path_b.exists() and path_c.exists()):
        print(f"  [X] {sb_name}: Faltan archivos, saltando.")
        continue

    print(f"  [OK] Procesando {sb_name}...")
    bins_encontrados.append(sb_name)

    try:
        # 1. Leer
        df_a = pd.read_csv(path_a, sep='\t', low_memory=False)
        df_b = pd.read_csv(path_b, sep='\t', low_memory=False)
        df_c = pd.read_csv(path_c, sep='\t', low_memory=False)

        # 2. Fusionar (Merge)
        df_merged = pd.merge(df_a, df_b, on=MERGE_KEYS, how='inner')
        df_merged = pd.merge(df_merged, df_c, on=MERGE_KEYS, how='inner')

        # 3. Etiquetar
        df_merged['SpeedBin'] = i
        
        all_dfs.append(df_merged)

    except Exception as e:
        print(f"  [ERROR] Fallo en {sb_name}: {e}")

if not all_dfs:
    raise FileNotFoundError("ERROR CRÍTICO: No se cargaron datos. Verifica las rutas.")

# Concatenar todo en el DataFrame Maestro
df = pd.concat(all_dfs, ignore_index=True)
raw_rows = len(df)
print(f"\n>> Carga completada. Filas crudas cargadas: {raw_rows:,}")


# ==============================================================================
# 3. LIMPIEZA Y CÁLCULO DE TASAS
# ==============================================================================
print(f"\n--- FASE 2: PROCESAMIENTO Y CÁLCULO ---")

# 1. Asegurar tipos numéricos
numeric_cols = ["Road", "Source", "Day", "Hour", "Distance", "SpeedBin", "ModelYr"] + list(pollutant_map.keys())
print("1. Normalizando tipos de datos numéricos...")
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 2. Filtrar Off-Network (Road != 1)
print("2. Eliminando Off-Network (Road ID 1)...")
if "Road" in df.columns:
    df_clean = df[df["Road"] != 1].copy()
else:
    df_clean = df.copy()

# 3. Convertir a Formato Largo (Melt)
print("3. Reestructurando datos (Melting)...")
long_rows = []
present_pollutants = [c for c in pollutant_map.keys() if c in df_clean.columns]

for col in present_pollutants:
    # Columnas necesarias
    cols_to_keep = ["Day", "Hour", "Road", "Source", "SpeedBin", "Distance", col]
    
    if all(c in df_clean.columns for c in cols_to_keep):
        sub = df_clean[cols_to_keep].copy()
        sub = sub.rename(columns={col: "raw_value"})
        sub["Pollutant"] = pollutant_map[col] # Renombrar a nombre limpio
        sub["mass_g"] = sub["raw_value"]
        
        # Optimización memoria
        sub["Pollutant"] = sub["Pollutant"].astype('category')
        
        long_rows.append(sub[["Day", "Hour", "Road", "Source", "SpeedBin", "Pollutant", "mass_g", "Distance"]])

long_df = pd.concat(long_rows, ignore_index=True)

# 4. Limpieza de Nulos y Ceros
long_df["Distance"] = long_df["Distance"].fillna(0)
long_df["mass_g"] = long_df["mass_g"].fillna(0)
long_df = long_df.query("mass_g > 0 or Distance > 0") # Eliminar filas vacías

# 5. Agregación y Cálculo de Tasas (g/km)
print("4. Calculando tasas ponderadas (Suma Masa / Suma Distancia)...")
grp = long_df.groupby(["Day", "Hour", "Road", "Source", "SpeedBin", "Pollutant"], 
                      observed=True, as_index=False).agg(
    total_mass_g=("mass_g", "sum"),
    total_distance_km=("Distance", "sum")
)

# Fórmula: g/km (evitando división por cero)
grp["rate_per_km"] = np.where(grp["total_distance_km"] > 0,
                              grp["total_mass_g"] / grp["total_distance_km"],
                              0.0)

# ==============================================================================
# 4. PIVOTE Y GUARDADO FINAL
# ==============================================================================
print(f"\n--- FASE 3: GENERACIÓN DE FORMATO FINAL ---")

# Pivotar para tener contaminantes como columnas
df_pivot = grp.pivot_table(
    index=['Day', 'Hour', 'Road', 'Source', 'SpeedBin'],
    columns='Pollutant', 
    values='rate_per_km', 
    aggfunc='first'
)

df_pivot.reset_index(inplace=True)
df_pivot['Road'] = df_pivot['Road'].astype(str) # Road como texto para consistencia
df_pivot = df_pivot.fillna(0)

# Guardar
Path(BASE_INPUT_DIR).mkdir(parents=True, exist_ok=True)
df_pivot.to_parquet(OUTPUT_PIVOTED_PARQUET, index=False)

# ==============================================================================
# 5. RESUMEN DE SALIDA (OUTPUT REPORT)
# ==============================================================================
end_time = time.time()
duration = end_time - start_time

print("\n" + "="*60)
print("             RESUMEN DE PROCESO EXITOSO")
print("="*60)
print(f"Tiempo Total de Ejecución: {duration:.2f} segundos")
print("-" * 60)
print(f"1. Archivos Procesados:")
print(f"   - Bins encontrados: {len(bins_encontrados)} ({bins_encontrados[0]} ... {bins_encontrados[-1]})")
print(f"   - Filas crudas leídas: {raw_rows:,}")
print("-" * 60)
print(f"2. Datos Generados:")
print(f"   - Filas en archivo final: {len(df_pivot):,}")
print(f"   - Columnas (Contaminantes): {list(df_pivot.columns[5:])}") # Omitimos las 5 llaves
print("-" * 60)
print(f"3. Archivo Guardado:")
print(f"   - Ruta: {OUTPUT_PIVOTED_PARQUET}")
print(f"   - Formato: Parquet (Optimizado)")
print("="*60)

# Preview final
print("\nVista previa (Primeras 5 filas):")
print(df_pivot.head())

--- FASE 1: CARGA DE DATOS ---
Directorio base: D:\MOVES\Sensitivity Analysis RS14\RS14\Sensitivity Analysis Files\All SBs
Iniciando carga y unificación de 16 Speed Bins...
  [OK] Procesando SB01...
  [OK] Procesando SB02...
  [OK] Procesando SB03...
  [OK] Procesando SB04...
  [OK] Procesando SB05...
  [OK] Procesando SB06...
  [OK] Procesando SB07...
  [OK] Procesando SB08...
  [OK] Procesando SB09...
  [OK] Procesando SB10...
  [OK] Procesando SB11...
  [OK] Procesando SB12...
  [OK] Procesando SB13...
  [OK] Procesando SB14...
  [OK] Procesando SB15...
  [OK] Procesando SB16...

>> Carga completada. Filas crudas cargadas: 1,830,912

--- FASE 2: PROCESAMIENTO Y CÁLCULO ---
1. Normalizando tipos de datos numéricos...
2. Eliminando Off-Network (Road ID 1)...
3. Reestructurando datos (Melting)...
4. Calculando tasas ponderadas (Suma Masa / Suma Distancia)...

--- FASE 3: GENERACIÓN DE FORMATO FINAL ---

             RESUMEN DE PROCESO EXITOSO
Tiempo Total de Ejecución: 11.80 segundos
-